# Thermal Component Library

## Setup

### Internals

In [1]:
using ModelingToolkit, Plots, DifferentialEquations
using OrdinaryDiffEq
using Symbolics
mtk = ModelingToolkit
import ThermalSystem_Models as tsm

In [ ]:
#= Ports =#
@variables t
const D = Differential(t)

function thermal_dof(name)
    @variables T(t)=300  [description = "Temperature (K)"]
    @variables ds(t)=0.0 [description = "Entropy Flow Rate = dS/dt W/K"]
    ODESystem(Equation[], t,[T,ds],[]; name = name)
end

function mechanical_dof(name)
    @variables P(t)=101325  [description = "Pressure, (Pa)"]
    @variables dv(t)=0.0    [description = "Volume flow rate, m^3/s, = v*A"]
    ODESystem(Equation[], t,[T,ds],[]; name = name)
end

function energy_dof(name)
    @variables dE(t) = 0 [description = "Energy Rate (W)"]
    ODESystem(Equation[], t,[dE],[]; name = name)
end

function signal_dof(name)
    @variables sig(t) = 0   [description = "Signal (unitless)"]
    ODESystem(Equation[], t,[sig],[]; name = name)
end

function eConnect(pins...)
    eqns = [sum(pin -> pin.dE, pins) ~ 0.0]
end

function thermalConnect(pins...)
    eqs = [
        sum(pin -> pin.ds, pins) ~ 0.0, # Entropy flow rate
    ]
    for pin in pins[2:end]
        push!(eqs, pins[1].T ~ pin.T)   # p_1 = p_2, p_1 = p_3,...
    end
    eqs
end

### Components

In [ ]:
# Energy to thermal dof transformer
function e2t_transformer(name)
    e = energy_dof(:e)  # (+) e.De = energy into the system
    q = thermal_dof(:q)
    eqs = [e.dE ~ -q.T * q.ds]
    compose(ODESystem(eqs, t,[],[]; name = name),e,q)
end

# constant energy flow source
function constantEnergyFlowSource(name; eflow)
    e = energy_dof(:e)
    eqs = [e.dE ~ -eflow]
    compose(ODESystem(eqs,t,[],[]; name = name),e)
end

function thermalGround(name; temperature = 300)
    q = thermal_dof(:q)
    eqs = [q.T ~ temperature]
    compose(ODESystem(eqs, t,[],[]; name = name), q)
end

function pointMass(name; mass = 1.0, specific_heat = 10)
    @parameters m=mass cp=specific_heat
    C = m * cp
    q = thermal_dof(:q);
    eqs = [D(q.T) ~ (q.T * q.ds)/ C]
    compose(ODESystem(eqs, t, [], [m,cp],name = name), q)
end

function thermalResistor(name; UA = 1.0)
    @parameters R=1/UA

    qi = thermal_dof(:qi)
    qo = thermal_dof(:qo)

    eqs = [ qi.T*qi.ds ~ 1/R * (qi.T - qo.T),
            0 ~ qo.T*qo.ds +  qi.T*qi.ds]
    compose(ODESystem(eqs, t, [], [R], name = name), qi,qo)
end
